## pandas 与建模代码的结合

In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame({
    'x0':[1, 2, 3, 4, 5],
    'x1':[0.01, -0.01, 0.25, -4.1, 0.],
    'y':[-1.5, 0., 3.6, 1.3, -2.]
})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [2]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [3]:
data.values  # 转换为NumPy数组

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
df2 = pd.DataFrame(data.values, columns=['one', 'two', 'three'])
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


In [7]:
df3 = data.copy()
df3['strings'] = ['a', 'b', 'c', 'd', 'e']
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [8]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

In [9]:
model_cols = ['x0', 'x1']
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

In [10]:
data['category'] = pd.Categorical(['a', 'b', 'a', 'a', 'b'],
                                 categories=['a', 'b'])
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


In [12]:
dummies = pd.get_dummies(data.category, prefix='category')
data_with_dummies = data.drop('category', axis=1).join(dummies)
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


## 使用 Patsy 创建模型描述

y ~ x0 + x1

In [13]:
data = pd.DataFrame({
    'x0':[1, 2, 3, 4, 5],
    'x1':[0.01, -0.01, 0.25, -4.1, 0.],
    'y':[-1.5, 0., 3.6, 1.3, -2.]
})
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [14]:
import patsy

y, X = patsy.dmatrices('y ~ x0 + x1', data)
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [15]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [16]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [17]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

In [18]:
patsy.dmatrices('y ~ x0 + x1', data)[1]

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [19]:
coef, resid, _, _ = np.linalg.lstsq(X, y)
coef

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [21]:
coef = pd.Series(coef.squeeze(),
                index=X.design_info.column_names)
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

### Patsy 公式中的数据转换

In [22]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

In [24]:
new_data = pd.DataFrame({
    'x0':[6, 7, 8, 9],
    'x1':[3.1, -0.5, 0, 2.3],
    'y':[1, 2, 3, 4]
})
new_X = patsy.build_design_matrices([X.design_info], new_data)
new_X

[DesignMatrix with shape (4, 3)
   Intercept  x0  np.log(np.abs(x1) + 1)
           1   6                 1.41099
           1   7                 0.40547
           1   8                 0.00000
           1   9                 1.19392
   Terms:
     'Intercept' (column 0)
     'x0' (column 1)
     'np.log(np.abs(x1) + 1)' (column 2)]

In [25]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)  # 加和封装
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

### Patsy 与分类数据

In [26]:
data = pd.DataFrame({
    'key1':['a', 'a', 'b', 'b', 'a', 'b', 'a', 'b'],
    'key2':[0, 1, 0, 1, 0, 1, 0, 0],
    'v1':[1, 2, 3, 4, 5, 6, 7, 8],
    'v2':[-1, 0, 2.5, -0.5, 4.0, -1.2, 0.2, -1.7]
})
y, X = patsy.dmatrices('v2 ~ key1', data)
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

In [27]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

In [28]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

In [29]:
data['key2'] = data['key2'].map({0:'zero', 1:'one'})
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [30]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [31]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

## statsmodels 介绍

* 线性模型

* 线性混合效应模型

* 方差分析方法

* 时间序列过程和状态空间模型

* 广义矩量法

### 评估线性模型

In [36]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

np.random.randn(12345)

N = 100
X = np.c_[
    dnorm(0, 0.4, size=N),
    dnorm(0, 0.6, size=N),
    dnorm(0, 0.2, size=N)
]
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

X[:5]

array([[ 0.2836989 , -0.17632922,  0.10622423],
       [ 0.19217237,  1.4544659 ,  0.69877764],
       [-0.12982914, -0.47039581,  0.16662696],
       [ 0.25786645,  0.2091081 , -0.18398517],
       [-0.66092609,  0.64054481,  0.20688018]])

In [37]:
y[:5]

array([ 0.28411817,  1.48187462, -0.20458159,  0.3319844 ,  0.40719756])

In [38]:
X_modle = sm.add_constant(X)  # 将截距列添加到现有矩阵
X_modle[:5]

array([[ 1.        ,  0.2836989 , -0.17632922,  0.10622423],
       [ 1.        ,  0.19217237,  1.4544659 ,  0.69877764],
       [ 1.        , -0.12982914, -0.47039581,  0.16662696],
       [ 1.        ,  0.25786645,  0.2091081 , -0.18398517],
       [ 1.        , -0.66092609,  0.64054481,  0.20688018]])

In [39]:
model = sm.OLS(y, X)  # 最小二乘线性回归
results = model.fit()
results.params

array([0.06516407, 0.30397028, 0.42478209])

In [40]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.502
Model:                            OLS   Adj. R-squared:                  0.486
Method:                 Least Squares   F-statistic:                     32.54
Date:                Mon, 18 Feb 2019   Prob (F-statistic):           1.22e-14
Time:                        15:49:40   Log-Likelihood:                -24.655
No. Observations:                 100   AIC:                             55.31
Df Residuals:                      97   BIC:                             63.12
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0652      0.052      1.243      0.2

In [43]:
data = pd.DataFrame(X, 
                    columns=['col0', 'col1', 'col2'])
data['y'] = y
data[:5]

,col0,col1,col2,y
0,0.283699,-0.176329,0.106224,0.284118
1,0.192172,1.454466,0.698778,1.481875
2,-0.129829,-0.470396,0.166627,-0.204582
3,0.257866,0.209108,-0.183985,0.331984
4,-0.660926,0.640545,0.206880,0.407198


In [44]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()
results.params

Intercept   -0.046275
col0         0.075172
col1         0.312963
col2         0.416530
dtype: float64

In [45]:
results.tvalues

Intercept   -1.452071
col0         1.429752
col1         6.794689
col2         5.648114
dtype: float64

In [46]:
results.predict(data[:5])

0   -0.035888
1    0.714428
2   -0.133846
3   -0.038083
4    0.190681
dtype: float64

### 评估时间序列处理

In [47]:
init_x = 4

import random

values = [init_x, init_x]
N = 1000

b0 = 0.8
b1 = -0.4
noise = dnorm(0, 0.1, N)
for i in range(N):
    new_x = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_x)

In [48]:
MAXLAGS = 5
model = sm.tsa.AR(values)
results = model.fit(MAXLAGS)
results.params

array([-0.01409259,  0.83539696, -0.42543779, -0.03654583,  0.00971396,
       -0.0068043 ])

## scikit-learn 介绍

In [50]:
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')

train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


In [51]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [52]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [53]:
# 使用训练集数据中间值填充表中的空值
impute_value = train['Age'].median()
train['Age'] = train['Age'].fillna(impute_value)
test['Age'] = test['Age'].fillna(impute_value)

# 明确模型
train['IsFemale'] = (train['Sex'] == 'female').astype(int)
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

predictors = ['Pclass', 'IsFemale', 'Age']
X_train = train[predictors].values
X_test = test[predictors].values
y_train = train['Survived'].values
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [54]:
y_train[:5]

array([0, 1, 1, 1, 0])

In [66]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)
y_predict = model.predict(X_test)
y_predict[:10]

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0])

In [67]:
from sklearn.linear_model import LogisticRegressionCV

modle_cv = LogisticRegressionCV(10)  # 交叉验证
modle_cv.fit(X_train, y_train)

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


LogisticRegressionCV(Cs=10, class_weight=None, cv='warn', dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='warn', n_jobs=None, penalty='l2',
           random_state=None, refit=True, scoring=None, solver='lbfgs',
           tol=0.0001, verbose=0)

In [68]:
from sklearn.model_selection import cross_val_score

model = LogisticRegression(C=10)
scores = cross_val_score(model, X_train, y_train, cv=4)
scores

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.77232143, 0.80269058, 0.77027027, 0.78828829])

## 继续你的教育

* Introduction to Machine Learning with Python by Andreas Mueller and Sarah Guido（O’Reilly） 

* Python Data Science Handbook by Jake VanderPlas（O’Reilly） 

* Data Science from Scratch: First Principles with Python by Joel Grus（O’Reilly） 

* Python Machine Learning by Sebastian Raschka（Packt Publishing） 

* Hands-On Machine Learning with Scikit-Learn and TensorFlow by Aurélien Géron（O’Reilly）
